Reads mat-file created by Pinnacles Utilities and saves as netcdf.

In [ ]:
import numpy as np
import xarray as xr

from scipy.io import loadmat
from datetime import datetime
from pathlib import Path

In [ ]:
matfile = 

savefile = Path(matfile).with_suffix('.nc')

print(f'Will convert {matfile} and save as {savefile}')

In [ ]:
# Load matlab struct as a nested list of dicts
data_nl = loadmat(matfile, simplify_cells = True)
data_nl

In [ ]:
# Arrange data variables (and add dimensions)
info = {}
data= {}

for key in data_nl.keys():
    if key[:3]=='RDI':
        info[key] = data_nl[key]
    
    elif len(data_nl[key].shape) == 1:
        if key[:3] != 'Ser': # 1d ser-data is related to timestamp and coordinates, and will be treated separately
            data[key] = (('time'), data_nl[key])
           
    elif len(data_nl[key].shape) == 2:
        data[key] = (('time','range'), data_nl[key])
info

In [ ]:
# Extract time coordinate
Time = []
for i in range(data_nl['SerYear'].shape[0]):
    
    d = datetime(2000 + data_nl['SerYear'][i],
                 data_nl['SerMon'][i],
                 data_nl['SerDay'][i],
                 data_nl['SerHour'][i],
                 data_nl['SerMin'][i],
                 data_nl['SerSec'][i],
                 data_nl['SerHund'][i]*10000) 
    Time.append(np.datetime64(d))

In [ ]:
# Range coordinate
number_of_cells = data_nl['SerEA1cnt'].shape[1]
range_bins = info['RDIBin1Mid'] + np.array(list(range(number_of_cells)))*info['RDIBinSize']

In [ ]:
# Create xarray dataset
ds = xr.Dataset(data, coords={'time': Time, 'range': range_bins}, attrs = info)
ds

In [ ]:
# Save as netcdf
ds.to_netcdf(savefile)